In [137]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
import statsmodels.api as sm
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
import numpy
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import warnings
from pandas import datetime
from sklearn.metrics import mean_squared_error

In [138]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [139]:
#Mapping months to numeric values
months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'June','07':'July','08':'Aug','09':'Sept','10':'Oct','11':'Nov','12':'Dec'}

In [140]:
import pandas as pd

#Importing files from specific folders and list of holidays
bus_no='6080'                                                 #Route number - Change this value accordingly 
path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Up'       #Up Direction files folder Path - Change the path accordingly    
down_path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Down'      #Down Direction files folder path - Change the path accordingly
allfiles = glob.glob(path + "/*.csv")
down_allfiles=glob.glob(down_path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)    #Holiday list for 2018
Holiday_2017 = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays 2017.csv', names=Holiday_colnames)     #Holiday list for 2017
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']

# Up direction processing

In [141]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']
holiday_day_2017 = Holiday_2017['Day']
holiday_month_2017 = Holiday_2017['Month']

In [142]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
import datetime
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,23):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][29:]
    date = os.path.splitext(base)[0][26:28]

    month = os.path.splitext(base)[0][23:25]
    year = os.path.splitext(base)[0][18:22]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [144]:
up_pass=[]
for j in range(len(timeslot)):
    if(j==19):
        continue
    df[j].index = pd.DatetimeIndex(freq = "d", start = 0, periods=df[j].shape[0])
    series = df[j].copy()
    series.drop(['Time Slot','Trips','isHoliday','Day'],axis=1,inplace=True)
    # seasonal difference
    X = series.values
    model = sm.tsa.statespace.SARIMAX(series.Passengers, trend='n', order=(1,0,0), seasonal_order=(0,1,1,7), enforce_invertibility=False)
    model_fit = model.fit()
    #print model_fit.summary()
    forecast_values = model_fit.forecast(steps=1)
    if forecast_values[0]<=0:
        up_pass.append(0)
    else:
        up_pass.append(round(forecast_values[0]))
up_pass.append(0.0)
    
trips = []
for j in range(len(timeslot)):
    if(j==19):
        continue
    target = pd.Series([current_dow,isCurrDay_holiday,], index = ['Day','isHoliday'])
    target['Passengers'] = up_pass[j]
    df_try=df[j][['Day','isHoliday','Passengers']].copy()
    labels=df[j]['Trips'].copy()

    labels=np.asarray(labels,dtype=float)
    df_try=np.asarray(df_try,dtype=float)
    target=np.asarray(target,dtype=float)
    target=[target]
    print target
    model_trips=sm.tsa.statespace.SARIMAX(endog=labels,exog=df_try,trend='n', order=(1,0,0), seasonal_order=(0,1,1,7),enforce_stationarity=True, enforce_invertibility=False)
    model_trips = model_trips.fit()
    trip_forecast = model_trips.forecast(steps = 1,exog=target)
    print trip_forecast
    if trip_forecast[0]<=0:
        trip=0
    else:
        trip=round(trip_forecast[0])
    trips.append(trip)
trips.append(0.0)
    

[array([  4.,   0.,  15.])]
[ 0.66475627]
[array([ 4.,  0.,  6.])]
[ 0.91935769]
[array([  4.,   0.,  19.])]
[ 4.8519198]
[array([  4.,   0.,  33.])]
[ 6.2789739]
[array([  4.,   0.,  30.])]
[ 4.27268715]
[array([  4.,   0.,  32.])]
[ 3.82969955]
[array([  4.,   0.,  19.])]
[ 3.04816819]
[array([  4.,   0.,  23.])]
[ 2.86569757]
[array([  4.,   0.,  32.])]
[ 2.19532964]
[array([  4.,   0.,  34.])]
[ 2.13155741]
[array([  4.,   0.,  31.])]
[ 3.40855136]
[array([  4.,   0.,  54.])]
[ 3.74258411]
[array([  4.,   0.,  63.])]
[ 2.94657811]
[array([  4.,   0.,  70.])]
[ 2.72748576]
[array([   4.,    0.,  100.])]
[ 2.88458282]
[array([   4.,    0.,  139.])]
[ 3.34519369]
[array([   4.,    0.,  182.])]


C:\Users\Lenovo\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[ 3.77765407]
[array([   4.,    0.,  106.])]
[ 2.68890038]
[array([   4.,    0.,  111.])]
[ 2.90892898]


# Down Direction Processing 

In [146]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']

In [147]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}
#Obtaining the day and date details for current day 
now = datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,24):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1
        
for files in down_allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][31:]   
    date = os.path.splitext(base)[0][28:30]
    month = os.path.splitext(base)[0][25:27]
    year = os.path.splitext(base)[0][20:24]
    
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0
        df[j]['Trips'] = df[j]['Trips'].apply(pd.to_numeric)

In [148]:
down_trips=[]
down_pass=[]

for j in range(len(timeslot)):
    if(j==19):
        continue
    df[j].index = pd.DatetimeIndex(freq = "d", start = 0, periods=df[j].shape[0])
    series = df[j].copy()
    series.drop(['Time Slot','Trips','isHoliday','Day'],axis=1,inplace=True)
    # seasonal difference
    X = series.values
    model = sm.tsa.statespace.SARIMAX(series.Passengers, trend='n', order=(1,0,0), seasonal_order=(0,1,1,7), enforce_invertibility=False)
    model_fit = model.fit()
    #print model_fit.summary()
    forecast_values = model_fit.forecast(steps=1)
    if forecast_values[0]<=0:
        down_pass.append(0)
    else:
        down_pass.append(round(forecast_values[0]))
down_pass.append(0.0)
    
for j in range(len(timeslot)): 
    if(j==19):
        continue
    target = pd.Series([current_dow,isCurrDay_holiday,], index = ['Day','isHoliday'])
    target['Passengers'] = down_pass[j]
    df_try=df[j][['Day','isHoliday','Passengers']].copy()
    labels=df[j]['Trips'].copy()

    labels=np.asarray(labels,dtype=float)
    df_try=np.asarray(df_try,dtype=float)
    target=np.asarray(target,dtype=float)
    target=[target]
    print target
    if(j<19):    
        model_trips=sm.tsa.statespace.SARIMAX(endog=labels,exog=df_try,trend='n', order=(1,0,0), seasonal_order=(1,1,1,7),enforce_stationarity=True, enforce_invertibility=False)
        model_trips = model_trips.fit()
        trip_forecast = model_trips.forecast(steps = 1,exog=target)
        print trip_forecast
        if trip_forecast[0]<=0:
            trip=0
        else:
            trip=round(trip_forecast[0])
    else:
        trip=0
    down_trips.append(trip)
down_trips.append(0.0)
    

[array([ 4.,  0.,  6.])]
[ 0.52651893]
[array([  4.,   0.,  17.])]
[ 1.72628375]
[array([  4.,   0.,  85.])]
[ 5.89755479]
[array([   4.,    0.,  129.])]
[ 6.1635167]
[array([  4.,   0.,  95.])]


C:\Users\Lenovo\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[ 4.36288743]
[array([   4.,    0.,  100.])]
[ 4.3692434]
[array([  4.,   0.,  60.])]


C:\Users\Lenovo\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[ 3.15156685]
[array([  4.,   0.,  49.])]
[ 3.26711887]
[array([  4.,   0.,  40.])]
[ 2.83818977]
[array([  4.,   0.,  35.])]
[ 3.22206234]
[array([  4.,   0.,  31.])]
[ 2.59522937]
[array([  4.,   0.,  18.])]
[ 2.78064128]
[array([  4.,   0.,  25.])]
[ 2.96146131]
[array([  4.,   0.,  32.])]
[ 3.02662158]
[array([  4.,   0.,  28.])]
[ 3.24762863]
[array([  4.,   0.,  20.])]
[ 2.73599191]
[array([  4.,   0.,  23.])]
[ 3.71473192]
[array([  4.,   0.,  23.])]
[ 3.44936436]
[array([  4.,   0.,  18.])]
[ 3.37466733]


In [149]:
print len(down_pass)
print len(down_trips)

20
20


In [150]:
#Saving Results into CSV files 
forecasted_trips = pd.DataFrame(columns=['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass'])
save_path=r'D:/Dharmit/Final Year Project/Forecasted Trips/'
for j in range(len(timeslot)):
    forecasted_trips= forecasted_trips.append(pd.Series([timeslot[j],trips[j],up_pass[j],down_trips[j],down_pass[j]], index = ['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass']), ignore_index=True)
filename ='Forecasted_trips_'+bus_no+'_'+str(now.month).zfill(2) +'_'+str(now.day)+'_'+str(day)+'_SARIMAX'
forecasted_trips.to_csv(save_path + filename +'.csv')